# Writing Data

The same bracket syntax used for reading also supports writes.
Values are sent to PowerWorld immediately.

```python
from esapp import PowerWorld
from esapp.components import Bus, Gen, Load

pw = PowerWorld("path/to/case.pwb")
```

In [ ]:
from esapp import PowerWorld
from esapp.components import Bus, Gen, Load, Branch
import numpy as np
import pandas as pd
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

The indexable interface supports four write patterns:

| Syntax | Behavior |
|---|---|
| `pw[Gen, "GenMW"] = 100.0` | Broadcast a **scalar** to every object |
| `pw[Gen, "GenMW"] = [100, 150, ...]` | Set **per-element** values (length must match) |
| `pw[Gen, ["GenMW", "GenStatus"]] = [100, "Closed"]` | Broadcast to **multiple fields** at once |
| `pw[Bus] = df` | **Bulk update** from a DataFrame (must include key columns) |

## Scalar Broadcast

A single value is applied to every object of that type.

In [ ]:
pw[Gen, "GenMW"] = 100.0

## Per-Element Values

A list or array whose length matches the number of objects sets
each one individually.

In [ ]:
pw[Gen, "GenMW"] = np.linspace(50, 200, len(pw[Gen]))

## Multiple Fields

Set several fields at once by passing a list of field names and a
matching list of values.

In [ ]:
pw[Gen, ["GenMW", "GenStatus"]] = [100.0, "Closed"]

## DataFrame Update

For targeted updates to specific objects, build a DataFrame that
includes the primary-key columns and the fields to change.

In [ ]:
updates = pd.DataFrame({
    "BusNum": pw[Bus]["BusNum"].head(3),
    "BusPUVolt": [1.02, 1.01, 0.99]
})
pw[Bus] = updates

## Read-Modify-Write

A common workflow: read existing values, transform them in pandas,
and write back. Here we scale all loads by 10%.

In [ ]:
loads = pw[Load, ["LoadMW", "LoadMVR"]]
loads["LoadMW"] *= 1.10
loads["LoadMVR"] *= 1.10
pw[Load] = loads